Übung: Secrets
----------------------

Am Beispiel von MySQL soll der Einsatz von Secrets Demonstriert werden.

Zuerst müssen die Secrets mittels `kubectl` erstellt werden:

In [ ]:
! kubectl create secret generic mysql-pass --from-literal=password=admin
! kubectl create secret generic mysql-user-pass --from-literal=password=secret

In [ ]:
! kubectl get secrets

Anschliessend sind die Umgebungsvariablen in der YAML Datei so zu setzen, dass sie auf die Secret Werte zugreifen

        env:
        - name: MYSQL_ROOT_PASSWORD
          valueFrom:
           secretKeyRef:
            name: mysql-pass
            key: password
            
        - name: MYSQL_PASSWORD
          valueFrom:
           secretKeyRef:
            name: mysql-user-pass
            key: password

Die anderen Werte, wie Username können fix in der YAML Datei gesetzt werden:

        env:
        - name: MYSQL_USER
          value: test
        - name: MYSQL_DATABASE
          value: test 
          
**Hinweis**: Die Informationen, welche Umgebungsvariablen wir setzen können, steht in der Dokumentation des Container Images [mysql](https://hub.docker.com/_/mysql)

In [ ]:
! kubectl apply -f 09-7-Secrets/mysql.yaml

Zum Testen verwenden wir [adminer](https://www.adminer.org/de/).

Der Server (Hostname) ist `mysql`, Username, Password und Datenbank ergibt sich aus vorheriger Konfiguration.

In [ ]:
! kubectl apply -f https://raw.githubusercontent.com/mc-b/duk/master/mysql/adminer.yaml
! echo "http://$(cat ~/work/server-ip):$(kubectl get service adminer -o=jsonpath='{ .spec.ports[0].nodePort }')/adminer"

- - -

### Security (Sicherheit)

Das es sich bei [Secrets](https://kubernetes.io/docs/concepts/configuration/secret/) um eine eigene Resource handelt, können wir diese separat Absichern. Z.B. Das Operating darf diese, trotz Zugriff auf das Dashbard, nicht sehen.

Wenn aber die Applikation/Datenbank diese als Klartext Umgebungvariablen braucht ist die Sicherheit, spätestens beim Wechseln in den Container, nicht mehr gegeben.


In [ ]:
! kubectl exec deployment/mysql -- printenv | grep MYSQL

#### Alternativen

Dann sind Alternativen gefragt, z.B.

* [Hashicorp Vault](https://www.vaultproject.io/)
* [Bitnami Sealed Secrets](https://github.com/bitnami-labs/sealed-secrets)

Quelle: [5 GitOps Best Practices](https://blog.argoproj.io/5-gitops-best-practices-d95cb0cbe9ff)

- - - 

Aufräumen

In [ ]:
! kubectl delete -f 09-7-Secrets/mysql.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/duk/master/mysql/adminer.yaml
! kubectl delete secret mysql-pass
! kubectl delete secret mysql-user-pass